In [17]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import load_model

from sklearn.model_selection import GridSearchCV
from sklearn import metrics

## Format Data

In [18]:
LOG_FILE = 'base_left-11.log'
OUTPUT_SIZE = 10
ACTIONS = {'Dash', 'Turn', 'Tackle', 'Kick'}

def fetch_data(log_file):
    X = []
    y = []
    i = 0

    with open(log_file) as f:
        lines = f.readlines()
    lines = [line.strip() for line in lines]

    while i < len(lines):
        words = lines[i].split()

        if words[3] == 'StateFeatures':
            action_exists = False
            for action in ACTIONS:
                if action in lines[i+1]:
                    action_exists = True

            if action_exists:
                state_features = get_feature_array(words[4:])
                X.append(state_features)

                action = get_action_array(lines[i+1].split()[4])
                y.append(action)

                i += 2
            else:
                i += 1
        else:
            i += 1
    
    return np.array(X), np.array(y)

def get_feature_array(features):
    feature_array = [float(feat) for feat in features]
    for feat in feature_array:
        assert feat >= -1.0 and feat <= 1.0
    return feature_array

def get_action_array(action):
    action_array = [0 for i in range(OUTPUT_SIZE)]
    param_str = action[action.index('(')+1:-1]

    if action[:4] == 'Dash':
        params = [float(param) for param in param_str.split(',')]
        action_array[0] = 1
        action_array[1] = params[0]
        action_array[2] = params[1]
    elif action[:4] == 'Turn':
        param = float(param_str)
        action_array[3] = 1
        action_array[4] = param
    elif action[:6] == 'Tackle':
        params = [float(param) for param in param_str.split(',')]
        action_array[5] = 1
        action_array[6] = params[0]
    elif action[:4] == 'Kick':
        params = [float(param) for param in param_str.split(',')]
        action_array[7] = 1
        action_array[8] = params[0]
        action_array[9] = params[1]

    return action_array

In [19]:
X_train, y_train = fetch_data(LOG_FILE)

In [20]:
print(X_train.shape)
print(y_train.shape)

(18010, 68)
(18010, 10)


## Make Keras Model

In [23]:

model = Sequential()

model.add(Dense(68, input_dim=68))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(10, activation='softmax'))    
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Printing a summary of the layers and weights in your model
model.summary()
    


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 68)                4692      
_________________________________________________________________
activation_11 (Activation)   (None, 68)                0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 68)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 100)               6900      
_________________________________________________________________
activation_12 (Activation)   (None, 100)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                1010      
Total params: 12,602
Trainable params: 12,602
Non-trainable params: 0
_________________________________________________________________


## Fit Model

In [24]:
fit = model.fit(X_train, y_train, batch_size=32, epochs=15, verbose=1)


Epoch 1/15
18010/18010 [==============================] - 2s 87us/step - loss: 875.4434 - acc: 0.7590
Epoch 2/15
18010/18010 [==============================] - 1s 55us/step - loss: 874.8255 - acc: 0.7786
Epoch 3/15
18010/18010 [==============================] - 1s 53us/step - loss: 874.7834 - acc: 0.7803
Epoch 4/15
18010/18010 [==============================] - 1s 53us/step - loss: 874.7787 - acc: 0.7802
Epoch 5/15
18010/18010 [==============================] - 1s 53us/step - loss: 874.8130 - acc: 0.7793
Epoch 6/15
18010/18010 [==============================] - 1s 53us/step - loss: 874.7129 - acc: 0.7843
Epoch 7/15
18010/18010 [==============================] - 1s 61us/step - loss: 874.6374 - acc: 0.7891
Epoch 8/15
18010/18010 [==============================] - 1s 59us/step - loss: 874.2422 - acc: 0.8352
Epoch 9/15
18010/18010 [==============================] - 1s 54us/step - loss: 873.9845 - acc: 0.8615: 1s - loss: 870.
Epoch 10/15
18010/18010 [==============================] - 1s 54u

## Save Model

In [25]:

model.save('basic_model.h5')  # creates a HDF5 file 'my_model.h5'

## Results

In [26]:
score = model.evaluate(X_train, y_train, verbose=0)
print('Training score:', score[0])
print('Training accuracy:', score[1])


Training score: 873.975846406
Training accuracy: 0.861632426403
